# Seminar 2


## Klasifikator slika tkiva transplantiranog srca

U sklopu seminara, cilj je bio kreirati klasifikator slika tkiva transplantiranog srca u 4 klase prema stupnju odbacivanja tkiva: 

    - 0R - nema naznaka odbacivanja
    - 1R - blago odbacivanje
    - 2R - umjereno odbacivanje
    - 3R - izrazito odbacivanje

Također, ovaj klasifikator je samo pokušaj klasifikacije za ovaj problem. Mala količina podataka može biti uzrok potencijalno niske točnosti.

In [1]:
from torchvision import transforms, datasets, models
import torch
from torch import optim, cuda
from torch.utils.data import DataLoader, sampler, random_split
import torch.nn as nn

from PIL import Image
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Prvo ćemo ispisati imena direktorija u kojima se nalaze ove 4 klase.

In [2]:
print(os.listdir('dataset'))

['0R', '1R', '2R', '3R']


Budući da je VGG16 treniran na slikama ImageNet skupa podataka, treba transformirati podatke na 224x224 veličinu.

In [3]:
image_transforms = {
    # Train uses data augmentation - commented part
    'train':
    transforms.Compose([
        # transforms.RandomResizedCrop(size=256, scale=(0.95, 1.0)),
        # transforms.RandomRotation(degrees=15),
        # transforms.ColorJitter(),
        # transforms.RandomHorizontalFlip(),
        transforms.Resize(size=2560),
        transforms.CenterCrop(size=224),  # Image net standards
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])  # Imagenet standards
    ]),
    'val':
    transforms.Compose([
        transforms.Resize(size=2560),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test':
    transforms.Compose([
        transforms.Resize(size=2560),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

Zatim, potrebno je rastaviti skup podataka u skup za učenje (train), skup za provjeru (validation), skup za testiranje (test).

In [4]:
batch_size = 128

all_data = datasets.ImageFolder(root='dataset')
train_data_len = int(len(all_data)*0.8)
valid_data_len = int((len(all_data) - train_data_len)/2)
test_data_len = int(len(all_data) - train_data_len - valid_data_len)
train_data, val_data, test_data = random_split(all_data, [train_data_len, valid_data_len, test_data_len])
train_data.dataset.transform = image_transforms['train']
val_data.dataset.transform = image_transforms['val']
test_data.dataset.transform = image_transforms['test']
print(len(train_data), len(val_data), len(test_data))

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

960 120 120


In [5]:
trainiter = iter(train_loader)
features, labels = next(trainiter)
print(features.shape, labels.shape)

torch.Size([128, 3, 224, 224]) torch.Size([128])


In [6]:
model = models.vgg16(pretrained=True)
model

C:\Users\Jakov\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Jakov\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\Jakov/.cache\torch\hub\checkpoints\vgg16-397923af.pth
100%|██████████| 528M/528M [02:30<00:00, 3.69MB/s] 


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [7]:
# Freeze early layers
for param in model.parameters():
    param.requires_grad = False

Nakon zamrzivanja predtreniranih slojeva mreže, sada je potrebno definirati klasifikatorski sloj koji će biti prilagođen ovom problemu.

In [8]:
n_classes = 4
n_inputs = model.classifier[6].in_features
# n_inputs will be 4096 for this case
# Add on classifier
model.classifier[6] = nn.Sequential(
    nn.Linear(n_inputs, 256),       # 256 ili 2560 ??
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(256, n_classes),
    nn.LogSoftmax(dim=1))

model.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Sequential(
    (0): Linear(in_features=4096, out_features=256, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.4, inplace=False)
    (3): Linear(in_features=256, out_features=4, bias=True)
    (4): LogSoftmax(dim=1)
  )
)

In [15]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
model.to("cpu")

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [16]:
model.class_to_idx = all_data.class_to_idx
model.idx_to_class = {
    idx: class_
    for class_, idx in model.class_to_idx.items()
}

list(model.idx_to_class.items())

[(0, '0R'), (1, '1R'), (2, '2R'), (3, '3R')]

Zatim, potrebno je naučiti model na skupu za učenje.

In [20]:
def train(model,
         criterion,
         optimizer,
         train_loader,
         val_loader,
         save_location,
         early_stop=3,
         n_epochs=20,
         print_every=2):
   
#Initializing some variables
  valid_loss_min = np.Inf
  stop_count = 0
  valid_max_acc = 0
  history = []
  model.epochs = 0
  
  #Loop starts here
  for epoch in range(n_epochs):
    
    train_loss = 0
    valid_loss = 0
    
    train_acc = 0
    valid_acc = 0
    
    model.train()
    ii = 0
    
    for data, label in train_loader:
      ii += 1
      data, label = data.to("cpu"), label.to("cpu")
      optimizer.zero_grad()
      output = model(data)
      
      loss = criterion(output, label)
      loss.backward()
      optimizer.step()
      
      # Track train loss by multiplying average loss by number of examples in batch
      train_loss += loss.item() * data.size(0)
      
      # Calculate accuracy by finding max log probability
      _, pred = torch.max(output, dim=1) # first output gives the max value in the row(not what we want), second output gives index of the highest val
      correct_tensor = pred.eq(label.data.view_as(pred)) # using the index of the predicted outcome above, torch.eq() will check prediction index against label index to see if prediction is correct(returns 1 if correct, 0 if not)
      accuracy = torch.mean(correct_tensor.type(torch.FloatTensor)) #tensor must be float to calc average
      train_acc += accuracy.item() * data.size(0)
      if ii%15 == 0:
        print(f'Epoch: {epoch}\t{100 * (ii + 1) / len(train_loader):.2f}% complete.')
      
    model.epochs += 1
    with torch.no_grad():
      model.eval()
      
      for data, label in val_loader:
        data, label = data.to("cpu"), label.to("cpu")
        
        output = model(data)
        loss = criterion(output, label)
        valid_loss += loss.item() * data.size(0)
        
        _, pred = torch.max(output, dim=1)
        correct_tensor = pred.eq(label.data.view_as(pred))
        accuracy = torch.mean(correct_tensor.type(torch.FloatTensor))
        valid_acc += accuracy.item() * data.size(0)
        
      train_loss = train_loss / len(train_loader.dataset)
      valid_loss = valid_loss / len(val_loader.dataset)
      
      train_acc = train_acc / len(train_loader.dataset)
      valid_acc = valid_acc / len(val_loader.dataset)
      
      history.append([train_loss, valid_loss, train_acc, valid_acc])
      
      if (epoch + 1) % print_every == 0:
        print(f'\nEpoch: {epoch} \tTraining Loss: {train_loss:.4f} \tValidation Loss: {valid_loss:.4f}')
        print(f'\t\tTraining Accuracy: {100 * train_acc:.2f}%\t Validation Accuracy: {100 * valid_acc:.2f}%')
        
      if valid_loss < valid_loss_min:
        torch.save(model.state_dict(), save_location)
        stop_count = 0
        valid_loss_min = valid_loss
        valid_best_acc = valid_acc
        best_epoch = epoch
        
      else:
        stop_count += 1
        
        # Below is the case where we handle the early stop case
        if stop_count >= early_stop:
          print(f'\nEarly Stopping Total epochs: {epoch}. Best epoch: {best_epoch} with loss: {valid_loss_min:.2f} and acc: {100 * valid_acc:.2f}%')
          model.load_state_dict(torch.load(save_location))
          model.optimizer = optimizer
          history = pd.DataFrame(history, columns=['train_loss', 'valid_loss', 'train_acc','valid_acc'])
          return model, history
        
  model.optimizer = optimizer
  print(f'\nBest epoch: {best_epoch} with loss: {valid_loss_min:.2f} and acc: {100 * valid_acc:.2f}%')
  
  history = pd.DataFrame(history, columns=['train_loss', 'valid_loss', 'train_acc', 'valid_acc'])
  return model, history


In [22]:
model, history = train(
    model,
    criterion,
    optimizer,
    train_loader,
    val_loader,
    save_location='./rejection.pt',
    early_stop=5,
    n_epochs=8,
    print_every=1)


Epoch: 0 	Training Loss: 0.0151 	Validation Loss: 0.0009
		Training Accuracy: 99.48%	 Validation Accuracy: 100.00%

Epoch: 1 	Training Loss: 0.0124 	Validation Loss: 0.0010
		Training Accuracy: 99.58%	 Validation Accuracy: 100.00%

Epoch: 2 	Training Loss: 0.0093 	Validation Loss: 0.0008
		Training Accuracy: 99.58%	 Validation Accuracy: 100.00%

Epoch: 3 	Training Loss: 0.0062 	Validation Loss: 0.0006
		Training Accuracy: 99.79%	 Validation Accuracy: 100.00%

Epoch: 4 	Training Loss: 0.0110 	Validation Loss: 0.0008
		Training Accuracy: 99.58%	 Validation Accuracy: 100.00%

Epoch: 5 	Training Loss: 0.0071 	Validation Loss: 0.0005
		Training Accuracy: 99.79%	 Validation Accuracy: 100.00%

Epoch: 6 	Training Loss: 0.0082 	Validation Loss: 0.0006
		Training Accuracy: 99.69%	 Validation Accuracy: 100.00%

Epoch: 7 	Training Loss: 0.0094 	Validation Loss: 0.0008
		Training Accuracy: 99.58%	 Validation Accuracy: 100.00%

Best epoch: 5 with loss: 0.00 and acc: 100.00%


In [23]:
history

,train_loss,valid_loss,train_acc,valid_acc
0,0.015059,0.000908,0.994792,1.0
1,0.012386,0.001003,0.995833,1.0
2,0.009295,0.000814,0.995833,1.0
3,0.006224,0.000582,0.997917,1.0
4,0.010974,0.000848,0.995833,1.0
5,0.007096,0.000507,0.997917,1.0
6,0.008197,0.000558,0.996875,1.0
7,0.009362,0.000774,0.995833,1.0


Može se primijetiti da je točnost izrazito visoka jer je model prethodno treniran na skupu ImageNet koji sadrži milijune slika raspoređenih u 1000 klasa.

Sada valja pogledati i točnost na skupu za testiranje.

In [24]:
def accuracy(model, test_loader, criterion):
  with torch.no_grad():
    model.eval()
    test_acc = 0
    for data, label in test_loader:
      data, label = data.to("cpu"), label.to("cpu")
      
      output = model(data)
      
      _, pred = torch.max(output, dim=1)
      correct_tensor = pred.eq(label.data.view_as(pred))
      accuracy = torch.mean(correct_tensor.type(torch.FloatTensor))
      test_acc += accuracy.item() * data.size(0)
      
    test_acc = test_acc / len(test_loader.dataset)
    return test_acc

In [25]:
model.load_state_dict(torch.load('./rejection.pt'))
test_acc = accuracy(model.to("cpu"), test_loader, criterion)
print(f'The model has achieved an accuracy of {100 * test_acc:.2f}% on the test dataset')

The model has achieved an accuracy of 100.00% on the test dataset


In [26]:
def evaluate(model, test_loader, criterion):
  
  classes = []
  acc_results = np.zeros(len(test_loader.dataset))
  i = 0

  model.eval()
  with torch.no_grad():
    for data, labels in test_loader:
      data, labels = data.to("cpu"), labels.to("cpu")
      output = model(data)
      
      for pred, true in zip(output, labels):
        _, pred = pred.unsqueeze(0).topk(1)
        correct = pred.eq(true.unsqueeze(0))
        acc_results[i] = correct.cpu()
        classes.append(model.idx_to_class[true.item()])
        i+=1
  
  results = pd.DataFrame({
      'class': classes,
      'results': acc_results    
  })
  results = results.groupby(classes).mean()

  return results

In [27]:
evaluate(model, test_loader, criterion)

,results
0R,1.0
1R,1.0
2R,1.0
3R,1.0


Na temelju rezultata, možemo zaključiti da se model prenaučio. Razlog tome je mali broj podataka koji nije mogao biti povećan zbog malo memorijskog prostora na računalu.

Međutim, ovo pokazuje da predtrenirani model poput VGG16 ima izrazitog potencijala za rješavanje ovakvog tipa problema.